# Import Neccesary packages

In [2]:
#All neccesary classes for project

#general
import seaborn as sns
import scipy.stats as ss
import pandas as pd
import numpy as np
import os
import pickle
import time
import tensorflow as tf
import shutil
from datetime import datetime
import os
import random
import sys
import threading


#for utility packages
from Utilities.utilities import import_data
from Utilities.utilities import DataFrameSelector
from Utilities.utilities import CategoricalEncoder

from functools import partial

#image manipulation
from PIL import Image as PI
from PIL import ImageFilter
from resizeimage import resizeimage
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#Import Custom Functions
from Utilities.model_builder import get_image
from Utilities.model_builder import get_file_lists
from Utilities.model_builder import parse_record
from Utilities.model_builder import get_batch
from Utilities.model_builder import build_iterator
from Utilities.models import log_dir_build
from Utilities.utilities import generate_image
from Utilities.utilities import generate_image_array
from Utilities.blur_tool import blur_images_in_directory
from Utilities.resize_tool import resize_images_in_directory
from Utilities.blur_tool import blur_and_print_image
from Utilities.utilities import import_data
from Utilities.utilities import DataFrameSelector
from Utilities.utilities import CategoricalEncoder
from Utilities.utilities import display_scores
from Utilities.bounded_file_label_extractor import get_files_and_labels
from Utilities.bounded_box_record_maker import process_bounded_image_files
from Utilities.build_image_data_notebook import process_dataset

#for preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.utils import shuffle

# Make TF Records

Links for labels, train, and validation are used to build tf_records

In [3]:
#Build tf_records
#Set Variables
validation_directory = "D:/Machine_Learning/Datasets/Cifar_80/test_data"
train_directory = "D:/Machine_Learning/Datasets/Cifar_80/train_data"
output_directory = "D:/Machine_Learning/Datasets/Cifar_80/tf_records"
labels_file = "D:/Machine_Learning/Datasets/Cifar_80/labels.txt"

num_threads = 2
num_shards = 2

In [4]:
#make validation records
process_dataset('validation', validation_directory, num_shards, labels_file, num_threads, output_directory)

#make validation records
process_dataset('train', train_directory, num_shards, labels_file, num_threads, output_directory)

Determining list of input files and labels from D:/Machine_Learning/Datasets/Cifar_80/test_data.
Found 8000 JPEG files across 80 labels inside D:/Machine_Learning/Datasets/Cifar_80/test_data.
Launching 2 threads for spacings: [[0, 4000], [4000, 8000]]
2019-01-09 15:35:27.423876 [thread 0]: Processed 1000 of 4000 images in thread batch.
2019-01-09 15:35:27.428877 [thread 1]: Processed 1000 of 4000 images in thread batch.
2019-01-09 15:35:28.797877 [thread 0]: Processed 2000 of 4000 images in thread batch.2019-01-09 15:35:28.797877 [thread 1]: Processed 2000 of 4000 images in thread batch.

2019-01-09 15:35:30.195876 [thread 0]: Processed 3000 of 4000 images in thread batch.
2019-01-09 15:35:30.197877 [thread 1]: Processed 3000 of 4000 images in thread batch.
2019-01-09 15:35:31.511878 [thread 0]: Processed 4000 of 4000 images in thread batch.
2019-01-09 15:35:31.525876 [thread 0]: Wrote 4000 images to D:/Machine_Learning/Datasets/Cifar_80/tf_records\validation-00000-of-00002
2019-01-09 

# Dev Area

Use this area to run tests on above